# 本ドキュメントについて
本ドキュメントは、主成分分析についてのレポートである。最初に主成分分析とは何かについてまとめた（１と２）。主成分分析のステップや実務上での使われ方について説明をすることで、実行や実際をイメージしやすいように努めた。その後、主成分分析についてpythonでの実装と数式での説明を行なっている（３）。一般的な主成分分析で使われる数式などでの説明はすでに多くがウェブサイトにあるため、このドキュメントではここで扱っている二元のケースを使ってできるだけ数式で証明にチャレンジしてみた。このチャレンジでは、四則演算レベルの数式で説明することで、やっている分析のステップが一体なんなのかを理解できるようにした。


## １.主成分分析についてまとめよ

### 一言で言うと
主成分分析は、「与えられたデータ情報の次元を圧縮する手法」の一つである。
実は日常生活でもデータを低次元化するということはよく行われている。

#### 例
例えば、肥満度を測るためのBMIという指標が存在する。

>BMI=体重÷(身長)^2

元のデータは(身長,体重)という2成分を持つデータであったのに対し、BMIはただ1成分の数値となっている。これが次元の圧縮であり、情報を削ぎ落としたにも関わらず、肥満度という特徴を表すのに十分な情報を持っている。このように、データに適切な処理を行えば、情報量の削減と特徴の抽出を同時に行う事ができる。また、人がイメージできないような高次元データも、3次元以下に圧縮すればグラフとして描画できるようになる、といったメリットもある。

#### モチベーションと実務での使われかた
実際は、特徴量があらかじめわかっている場合と言うのは非常に少ない。そこで、与えられたデータの傾向から自動的に特徴量を見つけ出し、その特徴を良く表す低次元データへと次元圧縮を行えないかというモチベーションが出てくる。(これはある種の機械学習であり、特に自動的に特徴量を見出すという点において、「教師なし学習」と分類されるものとも言えるらしい)。~~したがって、主成分分析はデータ解析において「とりあえずやってみること」の一つで、データの視覚化、線形回帰等多変量データ解析の前処理のために実行される。

#### 主成分分析のステップ
1. データの標準化:中心化、分散の基準化
2. 分散共分散行列の計算
3. 分散共分散行列を固有値固有ベクトル分解
4. 固有値の大きい方からいくつかの固有値固有ベクトルを取ってくる **→８０％程度を主成分!**
5. 主成分にデータを射影して視覚化および回帰などの処理を続行


## ２.主成分分析についてまとめよ(素人向け)

### 一言で言うと
統計学上のデータ解析手法のひとつで、たくさんのデータの種類を、より少ない種類に集約して説明したり、また他の種類であることを突き止めて要約する手法である。

###  ワールドカップに出場するには
あなたの所属する組織やチームはどういう人たちで成り立っているだろうか。
４年に一度開催されているサッカーのワールドカップ。ここに出場する人とはどういう人であるだろうか。


一つの答えは行動特性を捉えて、その行動特性ができる人のことである。


ヨーロッパはサッカー熱の高い国が多く、サッカー選手の中でも特に行動特性分析なども研究されていた。
実際、自己成熟性（冷静さや、ストレス耐性、ビジネスマナー）や、対人行動（親密さ、ムードメーカー性）、リーダー（方針の共有、チームへの情報共有、公平さ）の指標をはじめとする
２００以上のデータを分析がなされた。
結論は、ワールドカップに出場する選手は、「チームワークを重視する」ということだった。
サッカーは結果として、個人プレーではなくチームで行うものという当たり前といえば当たり前の結果が出ていたのだが、
改めて、たくさんのデータから、より少ない種類で説明されると、出場する選手もどのようなチームワークが良いのかを考えれるヒントにもなり、それがハイパフォーマンス（ワールドカップ出場）に繋がりやすくなるという理解を再認識しているとのことだった。

プロスポーツも統計学的な分析が進んできている。



## ３.主成分分析について数式を用いて説明せよ。Pythonで実装もせよ。

### 企業を規模の大きい順で理解してみる
以下に、８社をランダムにとってきた。この会社を規模がどこから大きいのかを調べて見たい。現在は、それぞれ規模の大きさを表す変数でそれぞれの会社を表している。株式時価総額（Market Cap)と資本金（Capital Stock)である。


In [1]:
#numpyとmatplotliとpandasを呼び出す
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

#dataframe from pandas
df = pd.read_csv('enterprise.csv', encoding='utf-8')
df

,entity,market_cap,capital_stock
0,ganho,1267,32
1,matsumotokiyoshi,137,137
2,asahkasei,952,824
3,kirin,1662,1278
4,aoki,139,111
5,shiseido,601,304
6,daiichiseimei,1412,1649
7,sharp,629,135


### ゴール
一般に主成分分析は、特徴量があらかじめわかっている場合と言うのは非常に少ないがここでは規模の大きさを表す２つの変数から、結論を出して行きたい。

数式に置き換え、企業規模をz、株式時価総額をx1、資本金をx2と置く。
時価総額と、資本金の両方を考慮して評価して行きたいというのは、それぞれのx1とx2に重み付けをすることになるので、 重み付けをaとして、以下の式とおける。

$z = a1x1 + a2x2$

例えば、重みが1のときに0番目のガンホーを評価すると

$1299 = (1)1267 +(1)32$

が得られる。

ガンホーだけで式を一般化すると、

$z = 1267a1 + 32a2$

が得られることになる。



この考え方を他の会社にも適用すると、それぞれ以下のような式が得られる。

ganho : $z = 1267a1 + 32a2$

matsumotokiyoshi : $z = 137a1 + 137a2$

asahkasei : $z = 952a1 + 824a2$

krin : $z = 1662a1 + 1278a2$

aoki : $z = 139a1 + 111a2$

shiseido : $z = 601a1 + 304a2$

daiichiseimei : $z = 1412a1 + 1649a2$

sharp : $z = 629a1 + 135a2$


このように出た方程式ではあるが、次に、これらのa1とa2をどのように決めれば良いかというのが次の問題となる。


### a1とa2をどのように決めれば良いか
ここでは、企業規模をzで判断したいということである。

つまり、zがもっともばらつくようにa1とa2を決めることを意味しており、もっと一般化した言い方では、zの分散を最大化するということになる。


### 平均
分散を求めるために、まずは平均を求める。


In [2]:
df.describe()

,market_cap,capital_stock
count,8.000000,8.000000
mean,849.875000,558.750000
std,571.240864,618.362296
min,137.000000,32.000000
25%,485.500000,129.000000
50%,790.500000,220.500000
75%,1303.250000,937.500000
max,1662.000000,1649.000000


### zの平均
今回のデータのそれぞれの平均は、849.875、558.75であるので、

$849.875a1+558.75a2$

となる。

### zの分散

分散は以下を解くことで出てくる。

$1/7(((1267a1+32a2)-(849.875a1+558.75a2))^2+((137a1+137a2)-(849.875a1+558.75a2))^2
+((952a1+824a2)-(849.875a1+558.75a2))^2+((1662a1+1278a2)-(849.875a1+558.75a2))^2
+((139a1+111a2)-(849.875a1+558.75a2))^2+((601a1+304a2)-(849.875a1+558.75a2))^2
+((1412a1+1649a2)-(849.875a1+558.75a2))^2+((629a1+135a2)-(849.875a1+558.75a2))^2)$




まとめると
z=
$326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)$



以下に、pandasのデータフレーム機能を使って、分散共分散行列にしてみるとあっていることがわかる。

In [3]:
df.cov()

,market_cap,capital_stock
market_cap,326316.125000,254327.392857
capital_stock,254327.392857,382371.928571


### 制約条件の確認


$z=326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)$


上記の計算式だけでは、　a1とa2に関する値は広すぎて決められない。



そのため前提条件として、a1とa2の関係がどのようになっているのかを考えてみると。


$a1^2+a2^2 = 1$


という条件を制約としておける。


## 条件付き極値問題

ここまでの数式と制約を鑑みると。



$a1^2+a2^2 = 1$

という制約のもとで、


$326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)$

を求めることになるというのは、


高校３年で習う、条件付き極値問題に置き換えることができた。



## ラグランジェ乗数法を使って解く


$a1^2+a2^2 = 1$



$326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)$


より、


　　　　


$g(a1,a2) = 326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)$


$f(a1,a2) = a1^2+a2^2 = 1$


とおくと、


　　　　


$ga1 = 2*326316.125a1 -2*254327.392857a2$


$ga2 = 2*382371.928571a2-2*254327.392857a1$


$fa1 = 2a1$


$fa2 = 2a2$



　　　　

したがって、


$326316.125a1-254327.392857a2-\lambda*a1=0$


$-254327.392857a1+382371.928571a2-\lambda*a2=0$

を解くことになる。

　　　　　

行列式を用いて表現すると、

$
\left(	
      \begin{array}{cc}
      326316.125 & -254327.392857 \\
      -254327.392857 & 382371.928571
      \end{array}
  \right)
  \left(
      \begin{array}{c}
      a1 \\
      a2
      \end{array}
  \right)
  =
  \lambda
  \left(
      \begin{array}{c}
      a1 \\
      a2
      \end{array}
  \right)
$  


　　　　　





ところで、

$326316.125a1-254327.392857a2-\lambda*a1=0$


$-254327.392857a1+382371.928571a2-\lambda*a2=0$

で、上式にa1、下式にa2をかけて、足してみると、  
　　　　
       
  

$326316.125a1^2+382371.928571a2^2-2(254327.392857a1a2)=\lambda$
となる。


　　　　　





この式がzの分散になった。


つまり、$\lambda$はzの分散だった。

そのため、固有値を求めることだと言い換えられることがわかった。（証明終わり）


## Pythonで固有値（と固有ベクトル）を求める

df.covを使って、配列に変換し、
その配列をnumpyで固有値と固有ベクトルを求めると上記の値とあってくることもわかる。

In [4]:
df.cov().as_matrix()

array([[ 326316.125     ,  254327.39285714],
       [ 254327.39285714,  382371.92857143]])

In [98]:
# 行列Aとして生成
A = df.cov().as_matrix()

# 行列Aの固有値・固有ベクトル
la, v = np.linalg.eig(A)
print("固有値：", la)
print("固有ベクトル：", v)

固有値： [  98476.90104264  610211.15252879]
固有ベクトル： [[-0.74482912 -0.66725526]
 [ 0.66725526 -0.74482912]]


## 固有値（と固有ベクトル）の大きいほうから

固有値の大きい方は610211であり、その場合の新しい軸となるa1は0.67、a2は0.74である。




In [103]:
#dataframe from pandas
df = pd.read_csv('enterprise_rank.csv', encoding='utf-8')
df

,entity,market_cap,capital_stock,0.67,0.74,z,Ranking
0,ganho,1267,32,848.89,23.68,872.57,4
1,matsumotokiyoshi,137,137,91.79,101.38,193.17,7
2,asahkasei,952,824,637.84,609.76,1247.60,3
3,kirin,1662,1278,1113.54,945.72,2059.26,2
4,aoki,139,111,93.13,82.14,175.27,8
5,shiseido,601,304,402.67,224.96,627.63,5
6,daiichiseimei,1412,1649,946.04,1220.26,2166.30,1
7,sharp,629,135,421.43,99.90,521.33,6


## 結論

上記の分析から、企業規模のナンバーワンはdaiichiseimeiであることがわかった。

株式時価総額では、kirin。資本金ではdaiichiseimeiだった。新しい軸でもdaiichiseimeiとなった。

In [101]:
## 無視してください。
##色々試し中。固有値を降順にソートするとか、、、
# tmp = {}
# for i, value in enumerate(la):
#     tmp[value] = i

# v_sorted = []
# for key in sorted(tmp.keys(), reverse=True):
#     v_sorted.append(v[tmp[key]])
# v_sorted = np.array(v_sorted)

# la_sorted = np.array(sorted(la, reverse=True))

# print("固有値：", la_sorted)
# print("固有ベクトル：", v_sorted)

固有値： [ 610211.15252879   98476.90104264]
固有ベクトル： [[ 0.66725526 -0.74482912]
 [-0.74482912 -0.66725526]]
